In [1]:
%%html
<style>
/* Any CSS style can go in here. */
.dataframe th {
    font-size: 12px;
}
.dataframe td {
    font-size: 12px;
}
</style>

(example-grid-search)=

# Tuning hyperparameters of a MetaLearner with ``MetaLearnerGridSearch``

Motivation
----------

We know that model selection and/or hyperparameter optimization (HPO) can
have massive impacts on the prediction quality in regular Machine
Learning. Yet, it seems that model selection and hyperparameter
optimization are  of substantial importance for CATE estimation with
MetaLearners, too, see e.g. [Machlanski et. al](https://arxiv.org/abs/2303.01412>).

However, model selection and HPO for MetaLearners look quite different from what we're used to from e.g. simple supervised learning problems. Concretely,

* In terms of a MetaLearners's option space, there are several levels
  to optimize for:

  1. The MetaLearner architecture, e.g. R-Learner vs DR-Learner
  2. The model to choose per base estimator of said MetaLearner architecture, e.g. ``LogisticRegression`` vs ``LGBMClassifier``
  3. The model hyperparameters per base model

*  On a conceptual level, it's not clear how to measure model quality
   for MetaLearners. As a proxy for the underlying quantity of
   interest one might look into base model performance, the R-Loss of
   the CATE estimates or some more elaborate approaches alluded to by
   [Machlanski et. al](https://arxiv.org/abs/2303.01412).

We think that HPO can be divided into two camps:

* Exploration of (hyperparameter, metric evaluation) pairs where the
  pairs do not influence each other (e.g. grid search, random search)

* Exploration of (hyperparameter, metric evaluation) pairs where the
  pairs do influence each other (e.g. Bayesian optimization,
  evolutionary algorithms); in other words, there is a feedback-loop between
  sample result and sample

In this example, we will illustrate the former and how one can make use of
{class}`~metalearners.grid_search.MetaLearnerGridSearch` for it. For the latter please
refer to the {ref}`example on model selection with optuna<example-optuna>`.

Loading the data
----------------

Just like in our {ref}`example on estimating CATEs with a MetaLearner
<example-basic>`, we will first load some experiment data:

In [2]:
import pandas as pd
from pathlib import Path
from git_root import git_root

df = pd.read_csv(git_root("data/learning_mindset.zip"))
outcome_column = "achievement_score"
treatment_column = "intervention"
feature_columns = [
    column for column in df.columns if column not in [outcome_column, treatment_column]
]
categorical_feature_columns = [
    "ethnicity",
    "gender",
    "frst_in_family",
    "school_urbanicity",
    "schoolid",
]
# Note that explicitly setting the dtype of these features to category
# allows both lightgbm as well as shap plots to
# 1. Operate on features which are not of type int, bool or float
# 2. Correctly interpret categoricals with int values to be
#    interpreted as categoricals, as compared to ordinals/numericals.
for categorical_feature_column in categorical_feature_columns:
    df[categorical_feature_column] = df[categorical_feature_column].astype("category")

Now that we've loaded the experiment data, we can split it up into
train and validation data:

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation, w_train, w_validation = train_test_split(
    df[feature_columns], df[outcome_column], df[treatment_column], test_size=0.25
)

Performing the grid search
--------------------------

We can run a grid search by using the {class}`~metalearners.grid_search.MetaLearnerGridSearch`
class. However, it's important to note that this class only supports a single MetaLearner
architecture at a time. If you're interested in conducting a grid search across multiple architectures,
it will require several grid searches.

Let's say we want to work with a {class}`~metalearners.DRLearner`. We can check the names of
the base models for this architecture with the following code:

In [4]:
from metalearners import DRLearner

print(DRLearner.nuisance_model_specifications().keys())
print(DRLearner.treatment_model_specifications().keys())

dict_keys(['propensity_model', 'variant_outcome_model'])
dict_keys(['treatment_model'])


We see that this MetaLearner contains three base models: ``"variant_outcome_model"``,
``"propensity_model"`` and ``"treatment_model"``.

Since our problem has a regression outcome, the ``"variant_outcome_model"`` should be a regressor.
The ``"propensity_model"`` and ``"treatment_model"`` are always a classifier and a regressor
respectively.

To instantiate the {class}`~metalearners.grid_search.MetaLearnerGridSearch` object we need to
specify the different base models to be used. Moreover, if we'd like to use non-default hyperparameters for a given base model, we need to specify those, too.

In this tutorial we test a ``LinearRegression`` and ``LGBMRegressor`` for the outcome model,
a ``LGBMClassifier`` and ``QuadraticDiscriminantAnalysis`` for the propensity model and a
``LGBMRegressor`` for the treatment model.

Finally we can define the hyperparameters to test for the base models using the ``param_grid``
parameter.

In [5]:
from metalearners.grid_search import MetaLearnerGridSearch
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

gs = MetaLearnerGridSearch(
    metalearner_factory=DRLearner,
    metalearner_params={"is_classification": False, "n_variants": 2},
    base_learner_grid={
        "variant_outcome_model": [LinearRegression, LGBMRegressor],
        "propensity_model": [LGBMClassifier, QuadraticDiscriminantAnalysis],
        "treatment_model": [LGBMRegressor],
    },
    param_grid={
        "variant_outcome_model": {
            "LGBMRegressor": {"n_estimators": [3, 5], "verbose": [-1]}
        },
        "treatment_model": {"LGBMRegressor": {"n_estimators": [1, 2], "verbose": [-1]}},
        "propensity_model": {
            "LGBMClassifier": {"n_estimators": [1, 2, 3], "verbose": [-1]}
        },
    },
)

Now we can call {meth}`~metalearners.grid_search.MetaLearnerGridSearch.fit` with the train
and validation data and can inspect the results ``DataFrame`` in ``results_``.

In [6]:
gs.fit(X_train, y_train, w_train, X_validation, y_validation, w_validation)
gs.results_

fit_time  \
metalearner propensity_model              propensity_model_n_estimators propensity_model_verbose variant_outcome_model variant_outcome_model_n_estimators variant_outcome_model_verbose treatment_model treatment_model_n_estimators treatment_model_verbose             
DRLearner   LGBMClassifier                1.0                           -1.0                     LinearRegression      NaN                                NaN                           LGBMRegressor   1                            -1                       0.362632   
                                                                                                                                                                                                        2                            -1                       0.388284   
                                          2.0                           -1.0                     LinearRegression      NaN                                NaN                           LGBMRegressor   1                            -1                       0.392640   
                                                                                                                                                                                                        2                            -1                       0.454654   
                                          3.0                           -1.0                     LinearRegression      NaN                                NaN                           LGBMRegressor   1                            -1                       0.451604   
                                                                                                                                                                                                        2                            -1                       0.512599   
                                          1.0                           -1.0                     LGBMRegressor         3.0                                -1.0                          LGBMRegressor   1                            -1                       0.752140   
                                                                                                                       5.0                                -1.0                          LGBMRegressor   1                            -1                       1.030002   
                                                                                                                       3.0                                -1.0                          LGBMRegressor   2                            -1                       0.813204   
                                                                                                                       5.0                                -1.0                          LGBMRegressor   2                            -1                       1.075202   
                                          2.0                           -1.0                     LGBMRegressor         3.0                                -1.0                          LGBMRegressor   1                            -1                       0.863883   
                                                                                                                       5.0                                -1.0                          LGBMRegressor   1                            -1                       1.065925   
                                                                                                                       3.0                                -1.0                          LGBMRegressor   2                            -1                       0.874774   
                                                                                                                       5.0                                -1.0                          LGBMRegressor   2                            -1                       1.115629  

Reusing base models
--------------------
In order to decrease the grid search runtime, it may sometimes be desirable to reuse some nuisance models.
We refer to our {ref}`example of model reusage <example-reuse>` for a more in depth explanation
on how this can be achieved, but here we'll show an example for the integration of model
reusage with {class}`~metalearners.grid_search.MetaLearnerGridSearch`.

We will reuse the ``"variant_outcome_model"`` of a {class}`~metalearners.TLearner` for
a grid search over the {class}`~metalearners.XLearner`.

In [7]:
from metalearners import TLearner, XLearner

tl = TLearner(
    False,
    2,
    LGBMRegressor,
    nuisance_model_params={"verbose": -1, "n_estimators": 20, "learning_rate": 0.05},
    n_folds=2,
)
tl.fit(X_train, y_train, w_train)

gs = MetaLearnerGridSearch(
    metalearner_factory=XLearner,
    metalearner_params={
        "is_classification": False,
        "n_variants": 2,
        "n_folds": 5, # The number of folds does not need to be the same as in the TLearner
        "fitted_nuisance_models": {
            "variant_outcome_model": tl._nuisance_models["variant_outcome_model"]
        },
    },
    base_learner_grid={
        "propensity_model": [LGBMClassifier],
        "control_effect_model": [LGBMRegressor, LinearRegression],
        "treatment_effect_model": [LGBMRegressor, LinearRegression],
    },
    param_grid={
        "propensity_model": {"LGBMClassifier": {"n_estimators": [5], "verbose": [-1]}},
        "treatment_effect_model": {
            "LGBMRegressor": {"n_estimators": [5, 10], "verbose": [-1]}
        },
        "control_effect_model": {
            "LGBMRegressor": {"n_estimators": [1, 3], "verbose": [-1]}
        },
    },
)

gs.fit(X_train, y_train, w_train, X_validation, y_validation, w_validation)
gs.results_

fit_time  \
metalearner propensity_model propensity_model_n_estimators propensity_model_verbose control_effect_model control_effect_model_n_estimators control_effect_model_verbose treatment_effect_model treatment_effect_model_n_estimators treatment_effect_model_verbose             
XLearner    LGBMClassifier   5                             -1                       LGBMRegressor        1.0                               -1.0                         LGBMRegressor          5.0                                 -1.0                            0.468590   
                                                                                                                                                                                               10.0                                -1.0                            0.626367   
                                                                                                         3.0                               -1.0                         LGBMRegressor          5.0                                 -1.0                            0.532225   
                                                                                                                                                                                               10.0                                -1.0                            0.691459   
                                                                                                         1.0                               -1.0                         LinearRegression       NaN                                 NaN                             0.293462   
                                                                                                         3.0                               -1.0                         LinearRegression       NaN                                 NaN                             0.366940   
                                                                                    LinearRegression     NaN                               NaN                          LGBMRegressor          5.0                                 -1.0                            0.427425   
                                                                                                                                                                                               10.0                                -1.0                            0.574789   
                                                                                                                                                                        LinearRegression       NaN                                 NaN                             0.252676   

                                                                                                                                                                                                                                                                   score_time  \
metalearner propensity_model propensity_model_n_estimators propensity_model_verbose control_effect_model control_effect_model_n_estimators control_effect_model_verbose treatment_effect_model treatment_effect_model_n_estimators treatment_effect_model_verbose               
XLearner    LGBMClassifier   5                             -1                       LGBMRegressor        1.0                               -1.0                         LGBMRegressor          5.0                                 -1.0                              0.046710   
                                                                                                                                                                                               10.0                                -1.0                              0.046410   
                                                                                                         3.0                               -1.0                         LGBMRegressor    

What if I run out of memory?
----------------------------

If you're conducting an optimization task over a large grid with a substantial dataset,
it is possible that memory usage issues may arise. To try to solve these, you can minimize
memory usage by adjusting your settings.

In that case you can set ``store_raw_results=False``, the grid search will then operate
with a generator rather than a list, significantly reducing memory usage.

If the ``results_ DataFrame`` is what you're after, you can simply set ``store_results=True``.
However, if you aim to iterate over the {class}`~metalearners.metalearner.MetaLearner` objects,
you can set ``store_results=False``. Consequently, ``raw_results_`` will become a generator
object yielding {class}`~metalearners.grid_search.GSResult`.

Further comments
-------------------
* We strongly recommend only reusing base models if they have been trained on
  exactly the same data. If this is not the case, some functionalities
  will probably not work as hoped for.